In [0]:
# Gold-zone
from azure.storage.blob import BlobServiceClient
from azure.core.exceptions import AzureError


azure_connection_string = "DefaultEndpointsProtocol=https;AccountName=datalakea04fb344bd6a3620;AccountKey=A34ScnKfIF7RVIEA3FiPD80PSQMi56XtrozPHuiXhOPlb4C/J3jXoSHRNCmqjhjt+nqjtoWevEhk+ASt3Ehffg==;EndpointSuffix=core.windows.net"


silver_container_name = "silver"
gold_container_name = "gold"


blob_service_client = BlobServiceClient.from_connection_string(azure_connection_string)


def copy_to_gold(file_name):
    try:
        
        source_blob = blob_service_client.get_blob_client(container=silver_container_name, blob=file_name)

        
        target_blob = blob_service_client.get_blob_client(container=gold_container_name, blob=file_name)

        
        target_blob.start_copy_from_url(source_blob.url)
        print(f"Arquivo {file_name} movido para a Gold Zone.")
    except AzureError as e:
        print(f"Ocorreu um erro ao mover o arquivo para a Gold Zone: {e}")


files_to_process = [
    "clientes.csv",
    "imovel.csv",
    "cobertura.csv",
    "apoliceseguro.csv",
    "corretor.csv",
    "seguradora.csv"
]


for file_name in files_to_process:
    copy_to_gold(file_name)

Arquivo clientes.csv movido para a Gold Zone.
Arquivo imovel.csv movido para a Gold Zone.
Arquivo cobertura.csv movido para a Gold Zone.
Arquivo apoliceseguro.csv movido para a Gold Zone.
Arquivo corretor.csv movido para a Gold Zone.
Arquivo seguradora.csv movido para a Gold Zone.


In [0]:
from pyspark.sql import SparkSession

# Inicializar SparkSession
spark = SparkSession.builder.appName("SilverToGold").getOrCreate()

# Configurações de armazenamento Azure
storage_account_name = "datalakea04fb344bd6a3620"
sas_token = (
    "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T08:01:27Z&"
    "st=2024-12-02T00:01:27Z&spr=https&sig=2yF996fA%2FS1HCzmNIIU73E5%2BWcQ9pnKYzWfCzioZMzo%3D"
)

# Configurar acesso ao Azure Data Lake
spark.conf.set(
    f"fs.azure.sas.silver.{storage_account_name}.blob.core.windows.net",
    sas_token
)
spark.conf.set(
    f"fs.azure.sas.gold.{storage_account_name}.blob.core.windows.net",
    sas_token
)

# Caminhos de entrada e saída
silver_path = f"wasbs://silver@{storage_account_name}.blob.core.windows.net"
gold_path = f"wasbs://gold@{storage_account_name}.blob.core.windows.net"

# Tabelas a serem processadas
tables = ["cliente", "imovel", "cobertura", "apolice_seguro", "seguradora"]

# Função para processar uma tabela
def process_table(table_name):
    print(f"Processando tabela: {table_name}")
    try:
        # Leitura da camada Silver
        df = spark.read.format("delta").load(f"{silver_path}/{table_name}")
        print(f"Tabela {table_name} lida com sucesso da camada Silver.")
        
        # Escrita na camada Gold
        df.write.format("delta").mode("overwrite").save(f"{gold_path}/{table_name}")
        print(f"Tabela {table_name} salva com sucesso na camada Gold.")
    except Exception as e:
        print(f"Erro ao processar a tabela {table_name}: {e}")

# Processar todas as tabelas
for table in tables:
    process_table(table)

print("Processamento Silver -> Gold finalizado com sucesso!")


Processando tabela: cliente
Tabela cliente lida com sucesso da camada Silver.
Tabela cliente salva com sucesso na camada Gold.
Processando tabela: imovel
Tabela imovel lida com sucesso da camada Silver.
Tabela imovel salva com sucesso na camada Gold.
Processando tabela: cobertura
Tabela cobertura lida com sucesso da camada Silver.
Tabela cobertura salva com sucesso na camada Gold.
Processando tabela: apolice_seguro
Tabela apolice_seguro lida com sucesso da camada Silver.
Tabela apolice_seguro salva com sucesso na camada Gold.
Processando tabela: seguradora
Tabela seguradora lida com sucesso da camada Silver.
Tabela seguradora salva com sucesso na camada Gold.
Processamento Silver -> Gold finalizado com sucesso!


In [0]:
%sql
CREATE TABLE IF NOT EXISTS datalakea04fb344bd6a3620_obt (
    ano INT,
    mes VARCHAR(10),
    valor_total_das_apolices NUMERIC(38, 2) NOT NULL,
    total_de_ativos NUMERIC(38, 2) NOT NULL,
    total_de_passivos NUMERIC(38, 2) NOT NULL,
    numero_total_de_imoveis INT NOT NULL,
    apolices_finalizadas INT NOT NULL,
    numero_de_sinistros INT NOT NULL,
    numero_de_apolices_vendidas INT NOT NULL,
    valor_medio_de_premio NUMERIC(38, 6) NOT NULL,
    corretor_id INT, 
    seguradora_id INT, 
    cobertura_id INT
);


In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("SilverToGold").getOrCreate()

storage_account_name = "datalakea04fb344bd6a3620"
sas_token = (
    "sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-12-30T08:01:27Z&"
    "st=2024-12-02T00:01:27Z&spr=https&sig=2yF996fA%2FS1HCzmNIIU73E5%2BWcQ9pnKYzWfCzioZMzo%3D"
)

spark.conf.set(
    f"fs.azure.sas.gold.{storage_account_name}.blob.core.windows.net",
    sas_token
)

gold_path = f"wasbs://gold@{storage_account_name}.blob.core.windows.net"

apoliceseguro_df = spark.read.option("header", "true").csv(f"{gold_path}/apoliceseguro.csv")
clientes_df = spark.read.option("header", "true").csv(f"{gold_path}/clientes.csv")
cobertura_df = spark.read.option("header", "true").csv(f"{gold_path}/cobertura.csv")
corretor_df = spark.read.option("header", "true").csv(f"{gold_path}/corretor.csv")
imovel_df = spark.read.option("header", "true").csv(f"{gold_path}/imovel.csv")
seguradora_df = spark.read.option("header", "true").csv(f"{gold_path}/seguradora.csv")

apoliceseguro_df.write.format("delta").mode("overwrite").save(f"{gold_path}/apoliceseguro_delta")
clientes_df.write.format("delta").mode("overwrite").save(f"{gold_path}/clientes_delta")
cobertura_df.write.format("delta").mode("overwrite").save(f"{gold_path}/cobertura_delta")
corretor_df.write.format("delta").mode("overwrite").save(f"{gold_path}/corretor_delta")
imovel_df.write.format("delta").mode("overwrite").save(f"{gold_path}/imovel_delta")
seguradora_df.write.format("delta").mode("overwrite").save(f"{gold_path}/seguradora_delta")

apoliceseguro_delta_df = spark.read.format("delta").load(f"{gold_path}/apoliceseguro_delta")
clientes_delta_df = spark.read.format("delta").load(f"{gold_path}/clientes_delta")
cobertura_delta_df = spark.read.format("delta").load(f"{gold_path}/cobertura_delta")
corretor_delta_df = spark.read.format("delta").load(f"{gold_path}/corretor_delta")
imovel_delta_df = spark.read.format("delta").load(f"{gold_path}/imovel_delta")
seguradora_delta_df = spark.read.format("delta").load(f"{gold_path}/seguradora_delta")

apoliceseguro_delta_df.show()
clientes_delta_df.show()
cobertura_delta_df.show()
corretor_delta_df.show()
imovel_delta_df.show()
seguradora_delta_df.show()



+----------+----------+---------+-------------+-----------+-----------+------------+
|apolice_id|cliente_id|imovel_id|seguradora_id|corretor_id|data_inicio|data_termino|
+----------+----------+---------+-------------+-----------+-----------+------------+
|         1|         1|        1|            1|          1| 2022-09-13|  2022-06-20|
|         2|         2|        2|            2|          2| 2023-09-15|  2023-07-28|
|         3|         3|        3|            3|          3| 2023-02-20|  2024-04-22|
|         4|         4|        4|            4|          4| 2021-12-31|  2023-05-08|
|         5|         5|        5|            5|          5| 2022-01-12|  2024-01-06|
|         6|         6|        6|            6|          6| 2024-03-30|  2024-09-12|
|         7|         7|        7|            7|          7| 2023-12-26|  2023-10-19|
|         8|         8|        8|            8|          8| 2024-10-04|  2024-08-10|
|         9|         9|        9|            9|          9| 2022-

In [0]:

final_df = apoliceseguro_delta_df \
    .join(clientes_delta_df, "cliente_id") \
    .join(cobertura_delta_df, apoliceseguro_delta_df["seguradora_id"] == cobertura_delta_df["seguradora_id"]) \
    .join(corretor_delta_df, "corretor_id") \
    .join(imovel_delta_df, "imovel_id") \
    .join(seguradora_delta_df, "seguradora_id")

final_df.show()


+-------------+---------+-----------+----------+----------+-----------+------------+----------------+--------------------+----------------+--------------------+------------+--------------------+------+------------------+-------------+-----------+-----------------+---------------+-------------+--------------------+--------------------+----------+--------------------+-----------+------------+--------------------+--------------------+-------------------+----------------+
|seguradora_id|imovel_id|corretor_id|cliente_id|apolice_id|data_inicio|data_termino|    nome_cliente|    endereco_cliente|telefone_cliente|       email_cliente|cobertura_id|           descricao|premio|limite_indenizacao|seguradora_id|obrigatorio|    nome_corretor|numero_registro|seguradora_id|   telefone_corretor|      email_corretor|cliente_id|     endereco_imovel|tipo_imovel|valor_imovel|     nome_seguradora| endereco_seguradora|telefone_seguradora|email_seguradora|
+-------------+---------+-----------+----------+------

In [0]:
from pyspark.sql import functions as F

# Adicionando alias para os DataFrames para evitar ambiguidade
resultado_df = apoliceseguro_delta_df.alias('apoliceseguro') \
    .join(clientes_delta_df.alias('clientes'), "cliente_id") \
    .join(cobertura_delta_df.alias('cobertura'), "seguradora_id") \
    .join(corretor_delta_df.alias('corretor'), "corretor_id") \
    .join(imovel_delta_df.alias('imovel'), "imovel_id") \
    .join(seguradora_delta_df.alias('seguradora'), "seguradora_id") \
    .withColumn("ano", F.year("apoliceseguro.data_inicio")) \
    .withColumn("mes", F.month("apoliceseguro.data_inicio")) \
    .groupBy("ano", "mes") \
    .agg(
        F.count("imovel.imovel_id").alias("numero_total_de_imoveis"),
        F.count("apoliceseguro.apolice_id").alias("apolices_finalizadas"),
        F.avg("cobertura.premio").alias("valor_medio_de_premio")
    )


In [0]:
resultado_df.show(10)


+----+---+-----------------------+--------------------+---------------------+
| ano|mes|numero_total_de_imoveis|apolices_finalizadas|valor_medio_de_premio|
+----+---+-----------------------+--------------------+---------------------+
|2022| 10|                    238|                 238|    4938.088235294118|
|2024|  7|                    279|                 279|     5070.79211469534|
|2023|  9|                    258|                 258|    5122.763565891473|
|2023|  8|                    269|                 269|    4762.449814126394|
|2024|  3|                    280|                 280|    4903.421428571429|
|2023|  7|                    278|                 278|    4760.582733812949|
|2024|  5|                    293|                 293|    4978.037542662116|
|2022|  2|                    262|                 262|    5140.103053435115|
|2022|  7|                    283|                 283|    5134.628975265017|
|2021| 11|                    118|                 118|    5046.

In [0]:
resultado_df.write.format("csv").option("header", "true").save("/path/to/save/directory")
